In [40]:
import pandas as pd
import numpy as np

def create_main_fields(care_areas_df, main_field_size):
    """Generate main fields around each care area."""
    main_fields = []
    main_field_id = 0
    
    for _, care_area in care_areas_df.iterrows():
        # Calculate the center of the care area
        x_center = (care_area["Xmin"] + care_area["Xmax"]) / 2
        y_center = (care_area["Ymin"] + care_area["Ymax"]) / 2
        
        # Define the main field around the care area
        main_field = {
            "ID": main_field_id,
            "Xmin": x_center - main_field_size / 2,
            "Xmax": x_center + main_field_size / 2,
            "Ymin": y_center - main_field_size / 2,
            "Ymax": y_center + main_field_size / 2
        }
        main_fields.append(main_field)
        main_field_id += 1

    return pd.DataFrame(main_fields)

def calculate_overlap(care_area, main_field):
    """Calculate the overlapping area between a care area and a main field."""
    x_min_overlap = max(care_area["Xmin"], main_field["Xmin"])
    x_max_overlap = min(care_area["Xmax"], main_field["Xmax"])
    y_min_overlap = max(care_area["Ymin"], main_field["Ymin"])
    y_max_overlap = min(care_area["Ymax"], main_field["Ymax"])
    
    if x_min_overlap < x_max_overlap and y_min_overlap < y_max_overlap:
        overlap_rect = {
            "Xmin": x_min_overlap,
            "Xmax": x_max_overlap,
            "Ymin": y_min_overlap,
            "Ymax": y_max_overlap
        }
        return overlap_rect
    else:
        return None

def create_subfields(main_fields_df, care_areas_df, sub_field_size):
    """Generate subfields within the overlapping regions of main fields and care areas."""
    sub_fields = []
    subfield_id = 0
    
    for _, main_field in main_fields_df.iterrows():
        for _, care_area in care_areas_df.iterrows():
            overlap_rect = calculate_overlap(care_area, main_field)
            if overlap_rect:
                sub_x_min, sub_y_min = overlap_rect["Xmin"], overlap_rect["Ymin"]
                sub_x_max, sub_y_max = overlap_rect["Xmax"], overlap_rect["Ymax"]
                
                # Calculate number of subfields in each dimension within the overlap region
                num_x_subfields = int(np.ceil((sub_x_max - sub_x_min) / sub_field_size))
                num_y_subfields = int(np.ceil((sub_y_max - sub_y_min) / sub_field_size))
                
                for i in range(num_x_subfields):
                    for j in range(num_y_subfields):
                        sub_x = sub_x_min + i * sub_field_size
                        sub_y = sub_y_min + j * sub_field_size
                        
                        # Ensure the subfield size stays within the overlap bounds
                        sub_x_max_actual = min(sub_x + sub_field_size, sub_x_max)
                        sub_y_max_actual = min(sub_y + sub_field_size, sub_y_max)
                        
                        sub_field = {
                            "ID": subfield_id,
                            "Xmin": sub_x,
                            "Xmax": sub_x_max_actual,
                            "Ymin": sub_y,
                            "Ymax": sub_y_max_actual,
                            "Main Field ID": int(main_field['ID'])
                        }
                        sub_fields.append(sub_field)
                        subfield_id += 1
    
    return pd.DataFrame(sub_fields)

def main():
    try:
        # Load the care areas CSV file
        care_areas_file_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\CareAreas.csv"
        care_areas_df = pd.read_csv(care_areas_file_path, names=["ID", "Xmin", "Xmax", "Ymin", "Ymax"])

        # Load the metadata CSV file
        metadata_file_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\metadata.csv"
        metadata_df = pd.read_csv(metadata_file_path)

        # Extract main field size and subfield size from metadata
        main_field_size = metadata_df.iloc[0]["Main Field Size"]
        sub_field_size = metadata_df.iloc[0]["Sub Field Size"]

        # Create main fields around care areas
        main_fields_df = create_main_fields(care_areas_df, main_field_size)

        # Create subfields within the overlapping regions
        sub_fields_df = create_subfields(main_fields_df, care_areas_df, sub_field_size)

        # Ensure IDs are integers
        main_fields_df["ID"] = main_fields_df["ID"].astype(int)
        sub_fields_df["ID"] = sub_fields_df["ID"].astype(int)
        sub_fields_df["Main Field ID"] = sub_fields_df["Main Field ID"].astype(int)
        care_areas_df["ID"] = care_areas_df["ID"].astype(int)

        # Save DataFrames to CSV files
        main_field_output_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\MainFields.csv"
        main_fields_df.to_csv(main_field_output_path, header=False, index=False)

        sub_field_output_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\SubFields.csv"
        sub_fields_df.to_csv(sub_field_output_path, header=False, index=False)

        # Print the resulting DataFrames
        print("Main Fields DataFrame:")
        print(main_fields_df)
        print("\nSub Fields DataFrame:")
        print(sub_fields_df)
    
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Main Fields DataFrame:
      ID   Xmin   Xmax   Ymin   Ymax
0      0  -49.0   51.0  -49.0   51.0
1      1  -49.0   51.0   -7.0   93.0
2      2  -49.0   51.0   35.0  135.0
3      3  -49.0   51.0   77.0  177.0
4      4  -49.0   51.0  119.0  219.0
..   ...    ...    ...    ...    ...
237  237  833.0  933.0  203.0  303.0
238  238  833.0  933.0  245.0  345.0
239  239  833.0  933.0  287.0  387.0
240  240  833.0  933.0  329.0  429.0
241  241  833.0  933.0  371.0  471.0

[242 rows x 5 columns]

Sub Fields DataFrame:
        ID   Xmin  Xmax   Ymin  Ymax  Main Field ID
0        0    0.0     2    0.0     2              0
1        1    0.0     2   42.0    44              0
2        2   42.0    44    0.0     2              0
3        3   42.0    44   42.0    44              0
4        4    0.0     2    0.0     2              1
...    ...    ...   ...    ...   ...            ...
1979  1979  882.0   884  420.0   422            240
1980  1980  840.0   842  378.0   380            241
1981  1981  840.0 

In [50]:
import pandas as pd
import numpy as np

def create_main_fields(care_areas_df, main_field_size):
    """Generate main fields around care areas."""
    main_fields = []
    main_field_id = 0
    
    # Set bounds for the area of interest
    x_min_bound, x_max_bound = care_areas_df["Xmin"].min(), care_areas_df["Xmax"].max()
    y_min_bound, y_max_bound = care_areas_df["Ymin"].min(), care_areas_df["Ymax"].max()
    
    for _, care_area in care_areas_df.iterrows():
        x_center = (care_area["Xmin"] + care_area["Xmax"]) / 2
        y_center = (care_area["Ymin"] + care_area["Ymax"]) / 2

        x_min = max(x_center - main_field_size / 2, x_min_bound)
        x_max = min(x_center + main_field_size / 2, x_max_bound)
        y_min = max(y_center - main_field_size / 2, y_min_bound)
        y_max = min(y_center + main_field_size / 2, y_max_bound)

        if x_min < x_max and y_min < y_max:
            main_field = {
                "ID": main_field_id,
                "Xmin": x_min,
                "Xmax": x_max,
                "Ymin": y_min,
                "Ymax": y_max
            }
            main_fields.append(main_field)
            main_field_id += 1

    return pd.DataFrame(main_fields)

def calculate_overlap(care_area, main_field):
    """Calculate the overlapping area between a care area and a main field."""
    x_min_overlap = max(care_area["Xmin"], main_field["Xmin"])
    x_max_overlap = min(care_area["Xmax"], main_field["Xmax"])
    y_min_overlap = max(care_area["Ymin"], main_field["Ymin"])
    y_max_overlap = min(care_area["Ymax"], main_field["Ymax"])
    
    if x_min_overlap < x_max_overlap and y_min_overlap < y_max_overlap:
        overlap_rect = {
            "Xmin": x_min_overlap,
            "Xmax": x_max_overlap,
            "Ymin": y_min_overlap,
            "Ymax": y_max_overlap
        }
        return overlap_rect
    else:
        return None

def create_subfields(main_fields_df, care_areas_df, sub_field_size):
    """Generate subfields within the overlapping regions of main fields and care areas."""
    sub_fields = []
    subfield_id = 0
    
    for _, main_field in main_fields_df.iterrows():
        for _, care_area in care_areas_df.iterrows():
            overlap_rect = calculate_overlap(care_area, main_field)
            if overlap_rect:
                sub_x_min, sub_y_min = overlap_rect["Xmin"], overlap_rect["Ymin"]
                sub_x_max, sub_y_max = overlap_rect["Xmax"], overlap_rect["Ymax"]
                
                # Calculate number of subfields in each dimension within the overlap region
                num_x_subfields = int(np.ceil((sub_x_max - sub_x_min) / sub_field_size))
                num_y_subfields = int(np.ceil((sub_y_max - sub_y_min) / sub_field_size))
                
                for i in range(num_x_subfields):
                    for j in range(num_y_subfields):
                        sub_x = sub_x_min + i * sub_field_size
                        sub_y = sub_y_min + j * sub_field_size
                        
                        # Calculate the actual extent of the subfield
                        sub_x_max_actual = sub_x + sub_field_size
                        sub_y_max_actual = sub_y + sub_field_size
                        
                        # Ensure subfields start within the overlap and extend beyond if needed
                        if sub_x < sub_x_max and sub_y < sub_y_max:
                            sub_field = {
                                "ID": subfield_id,
                                "Xmin": sub_x,
                                "Xmax": sub_x_max_actual,
                                "Ymin": sub_y,
                                "Ymax": sub_y_max_actual,
                                "Main Field ID": int(main_field['ID'])
                            }
                            sub_fields.append(sub_field)
                            subfield_id += 1
    
    return pd.DataFrame(sub_fields)

def main():
    try:
        # Load the care areas CSV file
        care_areas_file_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\CareAreas.csv"
        care_areas_df = pd.read_csv(care_areas_file_path, names=["ID", "Xmin", "Xmax", "Ymin", "Ymax"])

        # Load the metadata CSV file
        metadata_file_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\metadata.csv"
        metadata_df = pd.read_csv(metadata_file_path)

        # Extract main field size and subfield size from metadata
        main_field_size = metadata_df.iloc[0]["Main Field Size"]
        sub_field_size = metadata_df.iloc[0]["Sub Field Size"]

        # Create main fields around care areas
        main_fields_df = create_main_fields(care_areas_df, main_field_size)

        # Create subfields within the overlapping regions
        sub_fields_df = create_subfields(main_fields_df, care_areas_df, sub_field_size)

        # Ensure IDs are integers
        main_fields_df["ID"] = main_fields_df["ID"].astype(int)
        sub_fields_df["ID"] = sub_fields_df["ID"].astype(int)
        sub_fields_df["Main Field ID"] = sub_fields_df["Main Field ID"].astype(int)
        care_areas_df["ID"] = care_areas_df["ID"].astype(int)

        # Save DataFrames to CSV files
        main_field_output_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\MainFields.csv"
        main_fields_df.to_csv(main_field_output_path, header=False, index=False)

        sub_field_output_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\SubFields.csv"
        sub_fields_df.to_csv(sub_field_output_path, header=False, index=False)

        # Print the resulting DataFrames
        print("Main Fields DataFrame:")
        print(main_fields_df)
        print("\nSub Fields DataFrame:")
        print(sub_fields_df)
    
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Main Fields DataFrame:
      ID   Xmin   Xmax   Ymin   Ymax
0      0    0.0   51.0    0.0   51.0
1      1    0.0   51.0    0.0   93.0
2      2    0.0   51.0   35.0  135.0
3      3    0.0   51.0   77.0  177.0
4      4    0.0   51.0  119.0  219.0
..   ...    ...    ...    ...    ...
237  237  833.0  884.0  203.0  303.0
238  238  833.0  884.0  245.0  345.0
239  239  833.0  884.0  287.0  387.0
240  240  833.0  884.0  329.0  422.0
241  241  833.0  884.0  371.0  422.0

[242 rows x 5 columns]

Sub Fields DataFrame:
        ID   Xmin    Xmax   Ymin    Ymax  Main Field ID
0        0    0.0    2.56    0.0    2.56              0
1        1    0.0    2.56   42.0   44.56              0
2        2   42.0   44.56    0.0    2.56              0
3        3   42.0   44.56   42.0   44.56              0
4        4    0.0    2.56    0.0    2.56              1
...    ...    ...     ...    ...     ...            ...
1979  1979  882.0  884.56  420.0  422.56            240
1980  1980  840.0  842.56  378.0  380.

In [52]:
#using grid approach to optmize the code by minimizing the overlap of main field


import pandas as pd
import numpy as np

def create_main_fields(care_areas_df, main_field_size):
    """Generate main fields using a grid-based approach."""
    main_fields = []
    main_field_id = 0
    
    # Define the bounds of the area of interest
    x_min_bound, x_max_bound = care_areas_df["Xmin"].min(), care_areas_df["Xmax"].max()
    y_min_bound, y_max_bound = care_areas_df["Ymin"].min(), care_areas_df["Ymax"].max()
    
    # Create a grid of main fields
    x_start = np.floor(x_min_bound / main_field_size) * main_field_size
    x_end = np.ceil(x_max_bound / main_field_size) * main_field_size
    y_start = np.floor(y_min_bound / main_field_size) * main_field_size
    y_end = np.ceil(y_max_bound / main_field_size) * main_field_size

    for x in np.arange(x_start, x_end, main_field_size):
        for y in np.arange(y_start, y_end, main_field_size):
            main_field = {
                "ID": main_field_id,
                "Xmin": x,
                "Xmax": x + main_field_size,
                "Ymin": y,
                "Ymax": y + main_field_size
            }
            main_fields.append(main_field)
            main_field_id += 1

    return pd.DataFrame(main_fields)

def calculate_overlap(care_area, main_field):
    """Calculate the overlapping area between a care area and a main field."""
    x_min_overlap = max(care_area["Xmin"], main_field["Xmin"])
    x_max_overlap = min(care_area["Xmax"], main_field["Xmax"])
    y_min_overlap = max(care_area["Ymin"], main_field["Ymin"])
    y_max_overlap = min(care_area["Ymax"], main_field["Ymax"])
    
    if x_min_overlap < x_max_overlap and y_min_overlap < y_max_overlap:
        overlap_rect = {
            "Xmin": x_min_overlap,
            "Xmax": x_max_overlap,
            "Ymin": y_min_overlap,
            "Ymax": y_max_overlap
        }
        return overlap_rect
    else:
        return None

def create_subfields(main_fields_df, care_areas_df, sub_field_size):
    """Generate subfields within the overlapping regions of main fields and care areas."""
    sub_fields = []
    subfield_id = 0
    
    for _, main_field in main_fields_df.iterrows():
        for _, care_area in care_areas_df.iterrows():
            overlap_rect = calculate_overlap(care_area, main_field)
            if overlap_rect:
                sub_x_min, sub_y_min = overlap_rect["Xmin"], overlap_rect["Ymin"]
                sub_x_max, sub_y_max = overlap_rect["Xmax"], overlap_rect["Ymax"]
                
                # Calculate number of subfields in each dimension within the overlap region
                num_x_subfields = int(np.ceil((sub_x_max - sub_x_min) / sub_field_size))
                num_y_subfields = int(np.ceil((sub_y_max - sub_y_min) / sub_field_size))
                
                for i in range(num_x_subfields):
                    for j in range(num_y_subfields):
                        sub_x = sub_x_min + i * sub_field_size
                        sub_y = sub_y_min + j * sub_field_size
                        
                        # Calculate the actual extent of the subfield
                        sub_x_max_actual = sub_x + sub_field_size
                        sub_y_max_actual = sub_y + sub_field_size
                        
                        # Ensure subfields start within the overlap and extend beyond if needed
                        if sub_x < sub_x_max and sub_y < sub_y_max:
                            sub_field = {
                                "ID": subfield_id,
                                "Xmin": sub_x,
                                "Xmax": sub_x_max_actual,
                                "Ymin": sub_y,
                                "Ymax": sub_y_max_actual,
                                "Main Field ID": int(main_field['ID'])
                            }
                            sub_fields.append(sub_field)
                            subfield_id += 1

                            # Print subfield details
                            print(f"Subfield ID: {subfield_id - 1}")
                            print(f"Main Field ID: {sub_field['Main Field ID']}")
                            print(f"Xmin: {sub_field['Xmin']}")
                            print(f"Xmax: {sub_field['Xmax']}")
                            print(f"Ymin: {sub_field['Ymin']}")
                            print(f"Ymax: {sub_field['Ymax']}")
                            print("-" * 40)
    
    return pd.DataFrame(sub_fields)

def main():
    try:
        # Load the care areas CSV file
        care_areas_file_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\CareAreas.csv"
        care_areas_df = pd.read_csv(care_areas_file_path, names=["ID", "Xmin", "Xmax", "Ymin", "Ymax"])

        # Load the metadata CSV file
        metadata_file_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\metadata.csv"
        metadata_df = pd.read_csv(metadata_file_path)

        # Extract main field size and subfield size from metadata
        main_field_size = metadata_df.iloc[0]["Main Field Size"]
        sub_field_size = metadata_df.iloc[0]["Sub Field Size"]

        # Create main fields using a grid-based approach
        main_fields_df = create_main_fields(care_areas_df, main_field_size)

        # Create subfields within the overlapping regions
        sub_fields_df = create_subfields(main_fields_df, care_areas_df, sub_field_size)

        # Ensure IDs are integers
        main_fields_df["ID"] = main_fields_df["ID"].astype(int)
        sub_fields_df["ID"] = sub_fields_df["ID"].astype(int)
        sub_fields_df["Main Field ID"] = sub_fields_df["Main Field ID"].astype(int)
        care_areas_df["ID"] = care_areas_df["ID"].astype(int)

        # Save DataFrames to CSV files
        main_field_output_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\MainFields.csv"
        main_fields_df.to_csv(main_field_output_path, header=False, index=False)

        sub_field_output_path = r"C:\Users\sreev\OneDrive\Desktop\college\kla_hack\Dataset-1\Dataset-1\2nd\SubFields.csv"
        sub_fields_df.to_csv(sub_field_output_path, header=False, index=False)

        # Print the resulting DataFrames
        print("Main Fields DataFrame:")
        print(main_fields_df)
        print("\nSub Fields DataFrame:")
        print(sub_fields_df)
    
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Subfield ID: 0
Main Field ID: 0
Xmin: 0.0
Xmax: 2.56
Ymin: 0.0
Ymax: 2.56
----------------------------------------
Subfield ID: 1
Main Field ID: 0
Xmin: 0.0
Xmax: 2.56
Ymin: 42.0
Ymax: 44.56
----------------------------------------
Subfield ID: 2
Main Field ID: 0
Xmin: 0.0
Xmax: 2.56
Ymin: 84.0
Ymax: 86.56
----------------------------------------
Subfield ID: 3
Main Field ID: 0
Xmin: 42.0
Xmax: 44.56
Ymin: 0.0
Ymax: 2.56
----------------------------------------
Subfield ID: 4
Main Field ID: 0
Xmin: 42.0
Xmax: 44.56
Ymin: 42.0
Ymax: 44.56
----------------------------------------
Subfield ID: 5
Main Field ID: 0
Xmin: 42.0
Xmax: 44.56
Ymin: 84.0
Ymax: 86.56
----------------------------------------
Subfield ID: 6
Main Field ID: 0
Xmin: 84.0
Xmax: 86.56
Ymin: 0.0
Ymax: 2.56
----------------------------------------
Subfield ID: 7
Main Field ID: 0
Xmin: 84.0
Xmax: 86.56
Ymin: 42.0
Ymax: 44.56
----------------------------------------
Subfield ID: 8
Main Field ID: 0
Xmin: 84.0
Xmax: 86.56
Ymin: